In [9]:
import material_interface as mi

ni_poscar = """Ni - primitive
1.0
   2.130422000	   0.000000000	   1.230000000
   0.710140800	   2.008582000	   1.230000000
   0.000000000	   0.000000000	   2.460000000
Ni
1
Selective dynamics
direct
   0.000000000    0.000000000    0.000000000 T T T Ni"""

graphene_poscar = """Graphene
1.0
   2.467291000	   0.000000000	   0.000000000
  -1.233645000	   2.136737000	   0.000000000
   0.000000000	   0.000000000	   7.803074000
C
2
direct
   0.000000000    0.000000000    0.000000000  C
   0.333333000    0.666667000    0.000000000  C
"""

from ase.build import surface, supercells
from ase.io import read, write
from ase.visualize import view
import io
import numpy as np

ni = mi.poscar_to_atoms(ni_poscar)
gr = mi.poscar_to_atoms(graphene_poscar)

print(ni.get_cell())
print(gr.get_cell())

ni_surface = surface(ni, (1, 1, 0), layers=3)
ni_surface.center(vacuum=10, axis=2)
ni_surface.wrap()
view(ni_surface * [3, 3, 1])

gr.center(vacuum=10, axis=2)
gr.wrap()
view(gr)
# ni_slab = supercells.make_supercell(ni_surface, [[3, 0, 0], [0, 3, 0], [0, 0, 1]])
# ni_slab.center(vacuum=10, axis=2)
# ni_slab.wrap()

# view(ni_slab)

print(mi.write_atoms_to_poscar(ni))

Cell([[2.130422, 0.0, 1.23], [0.7101408, 2.008582, 1.23], [0.0, 0.0, 2.46]])
Cell([[2.467291, 0.0, 0.0], [-1.233645, 2.136737, 0.0], [0.0, 0.0, 7.803074]])
Ni 
 1.0000000000000000
     2.1304219999999998    0.0000000000000000    1.2300000000000000
     0.7101408000000000    2.0085820000000001    1.2300000000000000
     0.0000000000000000    0.0000000000000000    2.4600000000000000
 Ni 
   1
Cartesian
  0.0000000000000000  0.0000000000000000  0.0000000000000000



In [10]:
# !pip install git+https://github.com/nmdl-mizo/interface_master.git@develop

In [24]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, convert_vector_index
from interfacemaster.symmetric_tilt import get_csl_twisted_graphenes
from numpy import array, dot, round, cross, eye, pi, ceil, column_stack, arange
from numpy.linalg import inv, det, norm
from gb_code.csl_generator import print_list
import pandas as pd

from ase.io import write
import os


def write_atoms_to_cif(atoms, name):
    temp_filename = f"{name}.cif"
    write(temp_filename, atoms, format="cif")

    with open(temp_filename, "r") as file:
        content = file.read()

    return content


ni_cif = write_atoms_to_cif(ni, "m1")
gr_cif = write_atoms_to_cif(gr, "m2")

interface = core("m1.cif", "m2.cif")

hkl_1 = get_primitive_hkl([1, 1, 1], interface.conv_lattice_1, interface.lattice_1)
hkl_2 = get_primitive_hkl([0, 0, 1], interface.conv_lattice_2, interface.lattice_2)

interface.parse_limit(du=1e-2, S=1e-2, sgm1=50, sgm2=50, dd=1e-2)

interface.search_one_position_2D(hkl_1, hkl_2, theta_range=90, dtheta=0.01, exact=False)
interface.compute_bicrystal_two_D(hkl_1, hkl_2, normal_ortho=True, lim=50, tol_ortho=1e-2, tol_integer=1e-3)

/Users/mat3ra/.pyenv/versions/3.11.4/lib/python3.11/site-packages/pymatgen/io/cif.py:1197: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


Warning!, this programme will rewrite the POSCAR file in this dir!
Congrates, we found an appx CSL!

U1 = 
[[ 0  0]
 [-4 -3]
 [ 7 -4]]; sigma_1 = 37

U2 = 
[[-3 -4]
 [ 4 -7]
 [ 0  0]]; sigma_2 = 37

D = 
[[ 0.9976693  -0.00699724  0.00420224]
 [ 0.00793899  0.99733634 -0.00349614]
 [-0.00189545  0.00512729  0.99900113]]

axis = [2.47055535 1.74694644 4.2791274 ] ; theta = 3.64

cell 1:
[[-3  0  0]
 [ 1 -4  3]
 [ 1  7  4]]
cell 2:
[[ 0 -3  4]
 [ 0  4  7]
 [-1  0  0]]


In [20]:
import material_interface as mi

settings = {
    "slab": {
        "miller:h": 1,
        "miller:k": 1,
        "miller:l": 0,
        "vacuum": 4,
        "number_of_layers": 3,
    },
    "interface": {"slab_v:matrix": [[-2, 11], [2, -11]], "layer_v:matrix": [[10, -5], [0, -20]], "distance": 2.0},
}
interface = mi.MaterialInterface(ni, gr, settings)
view(interface.structure)

LinAlgError: Singular matrix

In [111]:
# !pip install --upgrade pymatgen
import pymatgen
from pymatgen.analysis.interfaces.zsl import ZSLGenerator
from pymatgen.core.structure import Structure
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder

substrate = Structure.from_str(ni_poscar, fmt="poscar")
layer = Structure.from_str(graphene_poscar, fmt="poscar")

zsl = ZSLGenerator(max_area=50)
cib = CoherentInterfaceBuilder(
    substrate_structure=substrate, film_structure=layer, substrate_miller=(1, 1, 1), film_miller=(0, 0, 1), zslgen=zsl
)

cib._find_matches()
cib._find_terminations()

In [112]:
print(f"MATCHES: {len(cib.zsl_matches)}", cib.zsl_matches)
print(f"TERMINATIONS: {len(cib.terminations)}", cib.terminations)

MATCHES: 233 [ZSLMatch(film_sl_vectors=array([[ 2.467291,  0.      ,  0.      ],
       [ 1.233645, -2.136737, -0.      ]]), substrate_sl_vectors=array([[-2.130422 ,  0.       ,  1.23     ],
       [-1.4202812,  2.008582 ,  0.       ]]), film_vectors=array([[ 2.467291,  0.      ,  0.      ],
       [-1.233645,  2.136737,  0.      ]]), substrate_vectors=array([[-1.4202812,  2.008582 ,  0.       ],
       [-2.130422 ,  0.       ,  1.23     ]]), film_transformation=array([[1., 0.],
       [0., 1.]]), substrate_transformation=array([[1., 0.],
       [0., 1.]])), ZSLMatch(film_sl_vectors=array([[2.467291e+00, 0.000000e+00, 0.000000e+00],
       [1.000000e-06, 4.273474e+00, 0.000000e+00]]), substrate_sl_vectors=array([[-1.4202812,  2.008582 ,  0.       ],
       [ 2.8405628,  2.008582 , -2.46     ]]), film_vectors=array([[ 2.467291,  0.      ,  0.      ],
       [-1.233645,  2.136737,  0.      ]]), substrate_vectors=array([[-1.4202812,  2.008582 ,  0.       ],
       [-2.130422 ,  0.       ,

In [117]:
interfaces = cib.get_interfaces(("C_P6/mmm_2", "Ni_R-3m_1"))
interfaces_list = list(interfaces)
for interface in interfaces_list:
    print(interface.to(fmt="poscar"))

from ase.visualize import view

if len(interfaces_list) < 10:
    for interface in interfaces_list:
        poscar = interface.to(fmt="poscar")
        i = mi.poscar_to_atoms(poscar)
        view(i)
else:
    poscar = interfaces_list[0].to(fmt="poscar")
    i = mi.poscar_to_atoms(poscar)
    view(i * [3, 3, 1])

Ni1 C2
1.0
   2.4599995727812636    0.0000000000000000    0.0000000000000002
   1.2299995285143266    2.1304228448014668    0.0000000000000002
   0.0000000000000000    0.0000000000000000   22.0000000000000000
Ni C
1 2
direct
  -0.6666665690958107   -0.6666663069138890    0.4393939393939394 Ni
   0.0000000000000000    0.0000000000000000    0.5303030303030303 C
   0.3333330000000001    0.3333330000000000    0.5303030303030303 C

Ni2 C4
1.0
   2.4600000686580152    0.0000000000000000    0.0000000000000002
   0.0000015075059087    4.2608448307190869    0.0000000000000003
   0.0000000000000000    0.0000000000000000   22.0000000000000000
Ni C
2 4
direct
  -0.9999995914617944    0.3333332845479054    0.4393939393939394 Ni
  -0.4999995914617947    0.8333332845479056    0.4393939393939394 Ni
   0.0000000000000000    0.0000000000000000    0.5303030303030303 C
   0.4999999999999999    0.5000000000000000    0.5303030303030303 C
   0.9999995000000000    0.3333335000000000    0.5303030303030303 C
  